In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
#dataのインポート+シャッフル
data = pd.read_csv("./dataset/exp_data.csv",index_col = 0).sample(frac=1, random_state=1).reset_index(drop=True)
print(data.shape)
print(data.columns)

(425, 32)
Index(['class_a', 'class_b', 'relation', 'ans_yamasaki', 'ans_ueda',
       'ans_daisuke', 'ans_hosomi', 'ans_kazuma', 'ans_sugi', 'type', 'title',
       'source', 'sim_by_mcg', 'is_a_by_mcg', 'lexvec_sim', 'wiki2vec_sim',
       'wn_hu_average', 'wn_pu_average', 'wn_hu_best', 'wn_pu_best',
       'wn_sim_path', 'wn_sim_lch', 'wn_sim_wup', 'wn_sim_res', 'wn_sim_jcn',
       'wn_sim_lin', 'has_same_word', 'is_include_word', 'bert_sim',
       'lexvec_diff', 'wiki2vec_diff', 'bert_diff'],
      dtype='object')


In [34]:
#カラム配列定義
#特徴量カラム
feature_columns =  ["sim_by_mcg","is_a_by_mcg","lexvec_sim","wiki2vec_sim","wn_hu_average",
                "wn_pu_average","wn_sim_path","wn_sim_wup",
                "is_include_word","bert_sim","lexvec_diff","wiki2vec_diff","bert_diff"]


#通常の特徴量ラベル
X_normal_columns = ["sim_by_mcg","is_a_by_mcg","lexvec_sim","wiki2vec_sim","wn_hu_average",
                "wn_pu_average","wn_sim_path","wn_sim_wup","is_include_word","bert_sim"]

#ベクトル系特徴量ラベル
pca_di={"lexvec_diff":30,"wiki2vec_diff":30,"bert_diff":3}

#被験者実験データ
ans_columns = [ 'ans_ueda',
       'ans_daisuke', 'ans_kazuma', 'ans_sugi']

In [35]:

# 欠損値は最頻値補完を行う
def fill_na_mean(data):
    data.loc[:,"sim_by_mcg"] = data.loc[:,"sim_by_mcg"].fillna(data.loc[:,"sim_by_mcg"].mode()[0])
    data.loc[:,"is_a_by_mcg"] = data.loc[:,"is_a_by_mcg"].fillna(data.loc[:,"is_a_by_mcg"].mode()[0])
    data.loc[:,"wiki2vec_sim"] = data.loc[:,"wiki2vec_sim"].fillna(data.loc[:,"wiki2vec_sim"].mode()[0])
    data.loc[:,"wiki2vec_diff"] = data.loc[:,"wiki2vec_diff"].fillna(data.loc[:,"wiki2vec_diff"].mode()[0])
    return data
data = fill_na_mean(data)
#クラスラベルエンコーディング
from sklearn.preprocessing import LabelEncoder
def label_encorder(data):
    le = LabelEncoder()
    encoded = le.fit_transform(data['relation'].values)
    for ans_c in ans_columns:
        temp_label = le.transform(data[ans_c].values)
        data[ans_c] = temp_label
    data.loc[:,'relation_label'] = encoded
    print(str(data["relation"].unique())+","+str(le.transform(data["relation"].unique())))
    return data
data = label_encorder(data)

['s1' 'h2' 'h1' 's2' 'k'],[3 1 0 4 2]


In [36]:
#特徴量の選別 wn_sim_jcnは値がバグっているので使わない(infinityが入っている)
X_data = data.loc[:,feature_columns]
# yの定義
y = data.loc[:,"relation_label"].values.astype("int32")

In [37]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#diffラベルのpca_ssモデル trainとtest arrを返す
def ss_pca(train,pca_n):
    scaler = StandardScaler()
    train=scaler.fit_transform(train)
    pca = PCA(n_components = pca_n)
    train = pca.fit_transform(train)
    return train
#str_arrをarrに変換
def convert_to_arr(vec_str):
    vec_arr = None
    for s in vec_str:
        if vec_arr is None:
            vec_arr =np.array(s[1:-1].split(", ")).astype("float32")
        else:
            vec_arr = np.block([[vec_arr],[np.array(s[1:-1].split(", ")).astype("float32")]])
    return vec_arr

In [38]:
#通常系の特徴量と、vec系の特徴量に分割し、vecにはpcaを行って最後に結合する
X = X_data.loc[:,X_normal_columns].values.astype("float64").astype("float64")
for c_name, n in pca_di.items():
    vec_arr = convert_to_arr(X_data.loc[:,c_name])
    pca_arr = ss_pca(vec_arr,n)
    X = np.concatenate([X, pca_arr], 1)


In [39]:
import optuna

/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore', FutureWarning)
#1.目的関数の定義
def objective(trial):
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 1)
    max_depth = trial.suggest_int('max_depth', 2,30)
    n_estimators = trial.suggest_int('n_estimators', 50, 700)
    clf1 = XGBClassifier(learning_rate=learning_rate,
                         max_depth=max_depth,
                         n_estimators=n_estimators,)
    
    score = cross_val_score(clf1, X, y, cv=90)
    print(np.mean(score))
    return np.mean(score)

#2.Optunaオブジェクトを作成
study = optuna.create_study(direction='maximize')
#3.Optunaでパラメータ探索
study.optimize(objective, n_trials=200)

#4.ベストパラメータ表示
study.best_params


[I 2020-12-15 04:08:07,993] A new study created in memory with name: no-name-4dd7559c-abde-4abc-b2c5-ed11836af998
/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 82 members, which is less than n_splits=90.
  warnings.warn(("The least populated class in y has only %d"
[I 2020-12-15 04:10:23,805] Trial 0 finished with value: 0.6094444444444443 and parameters: {'learning_rate': 0.7698243216149967, 'max_depth': 12, 'n_estimators': 461, 'min_child_weight': 7, 'gamma': 2.101266975184555, 'subsample': 0.8895582055431803, 'eta': 0.10560273947209119}. Best is trial 0 with value: 0.6094444444444443.


0.6094444444444443


/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 82 members, which is less than n_splits=90.
  warnings.warn(("The least populated class in y has only %d"
[I 2020-12-15 04:13:12,323] Trial 1 finished with value: 0.6005555555555555 and parameters: {'learning_rate': 0.06979152029733966, 'max_depth': 15, 'n_estimators': 493, 'min_child_weight': 9, 'gamma': 4.780397687846904, 'subsample': 0.32460619100784216, 'eta': 0.4482349952072008}. Best is trial 0 with value: 0.6094444444444443.


0.6005555555555555


/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 82 members, which is less than n_splits=90.
  warnings.warn(("The least populated class in y has only %d"
[I 2020-12-15 04:18:12,784] Trial 2 finished with value: 0.5872222222222222 and parameters: {'learning_rate': 0.3532680992117426, 'max_depth': 15, 'n_estimators': 499, 'min_child_weight': 1, 'gamma': 4.641178701360049, 'subsample': 0.660942847850083, 'eta': 0.4362841273000544}. Best is trial 0 with value: 0.6094444444444443.


...

/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 82 members, which is less than n_splits=90.
  warnings.warn(("The least populated class in y has only %d"
[I 2020-12-15 07:47:39,206] Trial 198 finished with value: 0.6533333333333333 and parameters: {'learning_rate': 0.3256310269108397, 'max_depth': 20, 'n_estimators': 435, 'min_child_weight': 9, 'gamma': 0.7138793678490838, 'subsample': 0.031068407254677074, 'eta': 0.3011820950306963}. Best is trial 158 with value: 0.6766666666666666.


0.6533333333333333


/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 82 members, which is less than n_splits=90.
  warnings.warn(("The least populated class in y has only %d"
[I 2020-12-15 07:48:32,704] Trial 199 finished with value: 0.6555555555555556 and parameters: {'learning_rate': 0.297298702836636, 'max_depth': 17, 'n_estimators': 269, 'min_child_weight': 7, 'gamma': 0.4206546754906668, 'subsample': 0.07768076821895784, 'eta': 0.3209514543601146}. Best is trial 158 with value: 0.6766666666666666.


0.6555555555555556


{'learning_rate': 0.24154467569115426,
 'max_depth': 17,
 'n_estimators': 312,
 'min_child_weight': 6,
 'gamma': 0.6112578201865325,
 'subsample': 0.05131165419770442,
 'eta': 0.3199279452974703}